# Libraries

In [7]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# Importing Helper Functions
from helper_functions import drop_extraneous_col, save_df
# Recursive Feature Elimination with Cross-Validation
from sklearn.feature_selection import RFECV
# Time Series Split and GridSearchCV, where GridSearchCV is for hyperparameter tuning
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV, cross_validate, RandomizedSearchCV
# Pipeline
from sklearn.pipeline import Pipeline
# Standard Scalar
from sklearn.preprocessing import StandardScaler
# Confusion Matrix
from sklearn.metrics import confusion_matrix, make_scorer, recall_score, precision_score, accuracy_score, f1_score, roc_auc_score
# Logistic Regression, Ridge Classifier
from sklearn.linear_model import LogisticRegression, RidgeClassifier
# Random Forest
from sklearn.ensemble import RandomForestClassifier
# XGBoost
from xgboost import XGBClassifier
# Support Vector Machine (SVM)
from sklearn.svm import LinearSVC


# Using the Sliding Window Average DataFrame

In [8]:
training_df  = pd.read_csv('/Users/siddmittal/Documents/School/3B/MSCI-446-Project/csvs/rolling_average_5_day.csv')
drop_extraneous_col(training_df)
training_df

,team0,team1,winner,season,date,team0_encoded,team1_encoded,restDays_team0,restDays_team1,mp_rolling_team0,...,blk%_rolling_team1,tov%_rolling_team0,tov%_rolling_team1,ortg_rolling_team0,ortg_rolling_team1,drtg_rolling_team0,drtg_rolling_team1,ft/fga_rolling_team0,ft/fga_rolling_team1,team1_winner
0,HOU,MEM,MEM,2018,2017-10-28,29,28,0.0,1.0,240.0,...,8.56,14.62,13.88,107.76,106.94,101.62,101.52,0.2350,0.3258,1
1,CLE,NOP,NOP,2018,2017-10-28,6,30,2.0,1.0,240.0,...,10.30,14.56,14.98,111.52,106.46,110.56,109.54,0.2222,0.2284,1
2,PHO,POR,POR,2018,2017-10-28,24,17,2.0,1.0,240.0,...,10.88,15.82,11.82,97.08,115.18,113.24,100.62,0.2390,0.2568,1
3,PHI,DAL,PHI,2018,2017-10-28,5,27,2.0,1.0,240.0,...,5.68,15.42,11.76,99.38,100.78,106.62,110.18,0.1702,0.2058,0
4,SAS,IND,IND,2018,2017-10-29,26,7,1.0,3.0,240.0,...,6.66,12.78,12.66,104.44,111.82,101.60,113.66,0.2076,0.2278,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7903,MIL,BOS,BOS,2024,2024-03-20,10,2,2.0,1.0,240.0,...,12.90,9.52,7.76,123.72,133.44,125.26,111.12,0.2296,0.1402,1
7904,PHI,PHO,PHO,2024,2024-03-20,5,24,1.0,2.0,240.0,...,8.00,11.72,11.66,104.76,120.48,107.80,124.40,0.1580,0.1348,1
7905,MIA,CLE,MIA,2024,2024-03-20,11,6,1.0,1.0,240.0,...,7.20,9.82,12.02,106.04,115.26,107.38,118.06,0.1430,0.1680,0
7906,LAC,POR,LAC,2024,2024-03-20,22,17,2.0,1.0,240.0,...,8.22,12.54,14.36,117.64,109.26,125.26,120.46,0.2178,0.1768,0


# Splitting Dataframe into Train and Test

In [9]:
undesired_columns = ['team0', 'team1', 'winner', 'season', 'date', 'team1_winner']
# We decided to train from the 2018 season to the 2023 season
training_seasons = [2018,2019,2020,2021,2022,2023]
# Splitting the dataframe into train and test
X_train = training_df[training_df['season'].isin(training_seasons)].drop(undesired_columns, axis=1)
X_test = training_df[training_df['season'] == 2024].drop(undesired_columns, axis=1)
y_train = training_df[training_df['season'].isin(training_seasons)]['team1_winner']
y_test = training_df[training_df['season'] == 2024]['team1_winner']


In [10]:
# Double checking the shapes of the training and testing dataframes

print(f'Observations in X_train: {X_train.shape[0]}')
print(f'Observations in y_train: {y_train.shape[0]}')

print(f'Observations in X_test: {X_test.shape[0]}')
print(f'Observations in y_test: {y_test.shape[0]}')


Observations in X_train: 6967
Observations in y_train: 6967
Observations in X_test: 941
Observations in y_test: 941


# Scaling Features

In [11]:
std_scalar = StandardScaler()
X_train = std_scalar.fit_transform(X_train)
X_test = std_scalar.fit_transform(X_test) 

# Defining the Type of Cross Validation

In [12]:
# Define the type of cross validation
tscv = TimeSeriesSplit()

# Creating a Results DataFrame to Store Training and Validation Scores

In [13]:
results_df = pd.DataFrame(columns=['Model', 'Training Accuracy', 'Validation Accuracy',
                             'Training Precision', 'Validation Precision',
                             'Training Recall ', 'Validation Recall',
                             'Training F1', 'Validation F1',
                             'Training ROC_AUC', 'Validation ROC_AUC'])

# Logistic Regression

In [14]:
# Define the Logistic Regresion Model
model_lr = LogisticRegression(solver='saga', max_iter=5000, random_state=42)
# Define the Recursive Feature Elimination Cross Validation
model_rfecv_lr = RFECV(estimator = model_lr, cv=tscv, min_features_to_select=30, scoring='accuracy')

# Fitting the rfecv model to the data
model_rfecv_lr.fit(X_train, y_train)
# Transforming the the training dataset to only have the selected features
X_train_selected = model_rfecv_lr.transform(X_train)
# Define a grid of hyperparameters
param_grid = {
    'C': [0.01, 0.1, 1, 10, 100],
    'penalty': ['l1', 'l2']
}
scoring = {
    'accuracy': make_scorer(accuracy_score),
    'precision': make_scorer(precision_score),
    'recall': make_scorer(recall_score),
    'f1': make_scorer(f1_score),
    'roc_auc': make_scorer(roc_auc_score)
}

grid_search_lr = GridSearchCV(estimator=model_lr, param_grid=param_grid, cv=tscv, scoring=scoring, refit='accuracy', verbose=1, return_train_score=True)
grid_search_lr.fit(X_train_selected,y_train)

cv_results_lr = grid_search_lr.cv_results_

new_row_data = ['Logistic Regression']
for scorer in scoring:
    best_validation_score = cv_results_lr[f'mean_test_{scorer}'].max()
    i = list(cv_results_lr[f'mean_test_{scorer}']).index(best_validation_score)
    train_score = list(cv_results_lr[f'mean_train_{scorer}'])[i]
    new_row_data.extend([train_score,best_validation_score])

best_params_lr = grid_search_lr.best_params_
num_features_lr = model_rfecv_lr.n_features_

new_row_series = pd.Series(new_row_data, index=results_df.columns)
results_df = pd.concat([results_df, pd.DataFrame(new_row_series).T], axis=0, ignore_index=True)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


# Random Forest Classifier

In [15]:
# Define the RandomForestClassifier Model
model_rfc = RandomForestClassifier(random_state=42)
# Define the Recursive Feature Elimination Cross Validation
model_rfecv_rfc = RFECV(estimator = model_rfc, cv=tscv, min_features_to_select=30, scoring='accuracy')
# Fitting the rfecv model to the data
model_rfecv_rfc.fit(X_train, y_train)
# Transforming the the training dataset to only have the selected features
X_train_selected = model_rfecv_rfc.transform(X_train)
# Define a grid of hyperparameters
param_grid = {
    'n_estimators': [100, 200, 500],       # Number of trees in the forest.
    'max_depth': [None, 10, 20, 30],        # Maximum depth of the tree.
    'min_samples_split': [2, 5, 10],        # Minimum number of samples required to split an internal node.
    'min_samples_leaf': [1, 2, 4], 
}
scoring = {
    'accuracy': make_scorer(accuracy_score),
    'precision': make_scorer(precision_score),
    'recall': make_scorer(recall_score),
    'f1': make_scorer(f1_score),
    'roc_auc': make_scorer(roc_auc_score)
}

random_search_rfc = RandomizedSearchCV(estimator=model_rfc, n_iter=100, param_distributions=param_grid, cv=tscv, scoring=scoring, refit='accuracy', verbose=1, return_train_score=True)
random_search_rfc.fit(X_train_selected,y_train)

cv_results_rfc = random_search_rfc.cv_results_

new_row_data = ['Random Forest Classifier']
for scorer in scoring:
    best_validation_score = cv_results_rfc[f'mean_test_{scorer}'].max()
    i = list(cv_results_rfc[f'mean_test_{scorer}']).index(best_validation_score)
    train_score = list(cv_results_rfc[f'mean_train_{scorer}'])[i]
    new_row_data.extend([train_score,best_validation_score])

num_features_rfc = model_rfecv_rfc.n_features_
best_params_rfc = random_search_rfc.best_params_

new_row_series = pd.Series(new_row_data, index=results_df.columns)
results_df = pd.concat([results_df, pd.DataFrame(new_row_series).T], axis=0, ignore_index=True)

Fitting 5 folds for each of 100 candidates, totalling 500 fits


# XGBoost Classifier

In [16]:
# Define the XGB Classifier Model
model_xgb = XGBClassifier(objective='binary:logistic')
# Define the Recursive Feature Elimination Cross Validation
model_rfecv_xgb = RFECV(estimator = model_xgb, cv=tscv, min_features_to_select=30, scoring='accuracy')
# Fitting the rfecv model to the data
model_rfecv_xgb.fit(X_train, y_train)
# Transforming the the training dataset to only have the selected features
X_train_selected = model_rfecv_xgb.transform(X_train)
# Define a grid of hyperparameters
param_grid = {
    'n_estimators': [100, 200, 300, 400],
    'learning_rate': [0.01, 0.05, 0.1, 0.2],
    'max_depth': [3, 4, 5, 6, 7],
    'colsample_bytree': [0.3, 0.5, 0.7, 0.9, 1.0],
    'subsample': [0.6, 0.7, 0.8, 0.9, 1.0],
    'gamma': [0, 0.1, 0.2, 0.3, 0.4],
}
scoring = {
    'accuracy': make_scorer(accuracy_score),
    'precision': make_scorer(precision_score),
    'recall': make_scorer(recall_score),
    'f1': make_scorer(f1_score),
    'roc_auc': make_scorer(roc_auc_score)
}

random_search_xgb = RandomizedSearchCV(estimator=model_xgb, n_iter=100, param_distributions=param_grid, cv=tscv, scoring=scoring, refit='accuracy', verbose=1, return_train_score=True)
random_search_xgb.fit(X_train_selected,y_train)

cv_results_xgb = random_search_xgb.cv_results_

new_row_data = ['XGB Classifier']
for scorer in scoring:
    best_validation_score = cv_results_xgb[f'mean_test_{scorer}'].max()
    i = list(cv_results_xgb[f'mean_test_{scorer}']).index(best_validation_score)
    train_score = list(cv_results_xgb[f'mean_train_{scorer}'])[i]
    new_row_data.extend([train_score,best_validation_score])

num_features_xgb = model_rfecv_xgb.n_features_
best_params_xgb = random_search_xgb.best_params_

new_row_series = pd.Series(new_row_data, index=results_df.columns)
results_df = pd.concat([results_df, pd.DataFrame(new_row_series).T], axis=0, ignore_index=True)

Fitting 5 folds for each of 100 candidates, totalling 500 fits


# Support Vector Machine

In [17]:
# Define the Support Vector Machine Model
model_svm = LinearSVC(max_iter=5000, random_state=42, dual='auto')
# Define the Recursive Feature Elimination Cross Validation
model_rfecv_svm = RFECV(estimator = model_svm, cv=tscv, min_features_to_select=30, scoring='accuracy')
# Fitting the rfecv model to the data
model_rfecv_svm.fit(X_train, y_train)
# Transforming the the training dataset to only have the selected features
X_train_selected = model_rfecv_svm.transform(X_train)
# Define a grid of hyperparameters
param_grid = {
    'C': [0.1, 1, 10, 100],  # Regularization parameter
    'penalty': ['l1', 'l2'],  # Penalty
}
scoring = {
    'accuracy': make_scorer(accuracy_score),
    'precision': make_scorer(precision_score),
    'recall': make_scorer(recall_score),
    'f1': make_scorer(f1_score),
    'roc_auc': make_scorer(roc_auc_score)
}

grid_search_svm = GridSearchCV(estimator=model_svm, param_grid=param_grid, cv=tscv, scoring=scoring, refit='accuracy', verbose=1, return_train_score=True)
grid_search_svm.fit(X_train_selected,y_train)

cv_results_svm = grid_search_svm.cv_results_

new_row_data = ['Linear SVC']
for scorer in scoring:
    best_validation_score = cv_results_svm[f'mean_test_{scorer}'].max()
    i = list(cv_results_svm[f'mean_test_{scorer}']).index(best_validation_score)
    train_score = list(cv_results_svm[f'mean_train_{scorer}'])[i]
    new_row_data.extend([train_score,best_validation_score])

num_features_svm = model_rfecv_svm.n_features_
best_params_svm = grid_search_svm.best_params_

new_row_series = pd.Series(new_row_data, index=results_df.columns)
results_df = pd.concat([results_df, pd.DataFrame(new_row_series).T], axis=0, ignore_index=True)

Fitting 5 folds for each of 8 candidates, totalling 40 fits


/Users/siddmittal/.pyenv/versions/3.11.0/lib/python3.11/site-packages/sklearn/svm/_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/siddmittal/.pyenv/versions/3.11.0/lib/python3.11/site-packages/sklearn/svm/_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/siddmittal/.pyenv/versions/3.11.0/lib/python3.11/site-packages/sklearn/svm/_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/siddmittal/.pyenv/versions/3.11.0/lib/python3.11/site-packages/sklearn/svm/_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/siddmittal/.pyenv/versions/3.11.0/lib/python3.11/site-packages/sklearn/svm/_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/User

In [23]:
results_df

,Model,Training Accuracy,Validation Accuracy,Training Precision,Validation Precision,Training Recall,Validation Recall,Training F1,Validation F1,Training ROC_AUC,Validation ROC_AUC
0,Logistic Regression,0.638975,0.613781,0.658646,0.635029,0.927034,0.918632,0.734622,0.71832,0.615209,0.591555
1,Random Forest Classifier,0.992214,0.597933,1.0,0.613177,0.992141,0.827351,0.937847,0.698188,0.990752,0.568822
2,XGB Classifier,0.731828,0.600172,0.970976,0.621755,0.963524,0.934788,0.836611,0.720567,0.988134,0.573162
3,Linear SVC,0.640713,0.612231,0.655526,0.636818,0.796979,0.74859,0.71934,0.685538,0.611674,0.594223


In [24]:
print(best_params_lr)
print(best_params_rfc)
print(best_params_xgb)
print(best_params_svm)

{'C': 0.1, 'penalty': 'l1'}
{'n_estimators': 200, 'min_samples_split': 10, 'min_samples_leaf': 4, 'max_depth': 30}
{'subsample': 0.7, 'n_estimators': 200, 'max_depth': 4, 'learning_rate': 0.01, 'gamma': 0.2, 'colsample_bytree': 0.9}
{'C': 0.1, 'penalty': 'l1'}


In [25]:
print(num_features_lr)
print(num_features_rfc)
print(num_features_xgb)
print(num_features_svm)

62
67
54
62


In [22]:
save_df(results_df, "rolling_average_5_day_results.csv")